### Imports

In [1]:
import argparse
from inspect import currentframe, getframeinfo
from glob import glob
import numpy as np
import os
from os.path import join
import pandas as pd
from pathlib import Path
import pickle
import sys
import nibabel as nib
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from nistats.first_level_model import FirstLevelModel
import warnings
from utils.plot_utils import plot_design
from utils.firstlevel_utils import get_first_level_objs, make_first_level_obj, save_first_level_obj

/tmp/ipykernel_114841/935669864.py:14: UserWarning: 

 | Using Nistats with Nilearn versions >= 0.7.0 is redundant and potentially conflicting.
 | Nilearn versions 0.7.0 and up offer all the functionality of Nistats as well the latest features and fixes.
 | We strongly recommend uninstalling Nistats and using Nilearn's stats & reporting modules.

  from nistats.first_level_model import FirstLevelModel
/home/users/sjshim/miniconda3/envs/fmri_analysis/lib/python3.9/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


### Parse Arguments
These are not needed for the jupyter notebook, but are used after conversion to a script for production

- conversion command:
  - jupyter nbconvert --to script --execute 1stlevel_analysis.ipynb

In [2]:
parser = argparse.ArgumentParser(description='First Level Entrypoint script')
parser.add_argument('-data_dir', default='/data')
parser.add_argument('-derivatives_dir', default=None)
parser.add_argument('-fmriprep_dir', default=None)
parser.add_argument('-working_dir', default=None)
parser.add_argument('--subject_ids', nargs="+")
parser.add_argument('--tasks', nargs="+", help="Choose from ANT, CCTHot, discountFix,                                     DPX, motorSelectiveStop, stopSignal,                                     stroop, surveyMedley, twoByTwo, WATT3")
parser.add_argument('--rt', action='store_true')
parser.add_argument('--beta', action='store_true')
parser.add_argument('--n_procs', default=16, type=int)
parser.add_argument('--overwrite', action='store_true')
parser.add_argument('--quiet', '-q', action='store_true')
parser.add_argument('--design_matrix', '-dm', action='store_true')
parser.add_argument('--a_comp_cor', action='store_true')

if '-derivatives_dir' in sys.argv or '-h' in sys.argv:
    args = parser.parse_args()
else:
    args = parser.parse_args([])
#     args.tasks = ['discountFix', 'manipulationTask', #aim 2 tasks
#                   'motorSelectiveStop', 'stopSignal']
    
    # args.tasks = ['ANT', 'CCTHot', 'discountFix', 'DPX', #aim 1 tasks
    #               'motorSelectiveStop', 'stopSignal', 
    #               'stroop', 'twoByTwo', 'WATT3']
    
   
    #args.subject_ids = ['3010']
    args.rt=True
    args.a_comp_cor=True
    args.n_procs=1
    args.derivatives_dir = '/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS/derivatives'
    args.data_dir = '/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS'
    args.fmriprep_dir = '/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS/derivatives/SOBC_analysis'
    args.design_matrix=False

In [3]:
if not args.quiet:
    def verboseprint(*args, **kwargs):
        print(*args, **kwargs)
else:
    verboseprint = lambda *a, **k: None # do-nothing function

### Initial Setup

Organize paths and set parameters based on arguments

In [4]:
# Set Paths
derivatives_dir = args.derivatives_dir
if args.fmriprep_dir is None:
    fmriprep_dir = join(derivatives_dir, 'fmriprep')
else:
    fmriprep_dir = args.fmriprep_dir
data_dir = args.data_dir
first_level_dir = join(derivatives_dir,'1stlevel')
if args.working_dir is None:
    working_dir = join(derivatives_dir, '1stlevel_workingdir')
else:
    working_dir = join(args.working_dir, '1stlevel_workingdir')

# set tasks
if args.tasks is not None:
    tasks = args.tasks
else:
    #tasks = ['flanker']
    
    tasks = ['cuedTS', 'directedForgetting', 'flanker', 'goNogo', 
                'nBack', 'shapeMatching', 'spatialTS',
                'stopSignal', 'stopSignalWDirectedForgetting',
                'stopSignalWFlanker', 'directedForgettingWFlanker']
    
    # tasks = ['ANT', 'CCTHot', 'discountFix', 'DPX', 
    #               'motorSelectiveStop', 'stopSignal', 
    #               'stroop', 'twoByTwo', 'WATT3']
    

# list of subject identifiers
if not args.subject_ids:
    subjects = sorted([i.split("-")[-1] for i in glob(os.path.join(args.data_dir, '*')) if 'sub-' in i])
else:
    subjects = args.subject_ids
ses = []
for i in range(1, 10):
    ses.append('ses-0'+str(i))
for i in range(10, 13):
    ses.append('ses-'+str(i))
#removing pilot subject
subjects.remove('n01')
#removing s03 for brainmask issue
#subjects.remove('s03')
# other arguments
regress_rt = args.rt
beta_series = args.beta
n_procs = args.n_procs
# TR of functional images
TR = 1.49
a_comp_cor=args.a_comp_cor

In [5]:
print(first_level_dir)
print(subjects)
print(fmriprep_dir)
print(ses)

/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS/derivatives/1stlevel
['s03', 's10', 's19', 's29', 's43']
/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS/derivatives/SOBC_analysis
['ses-01', 'ses-02', 'ses-03', 'ses-04', 'ses-05', 'ses-06', 'ses-07', 'ses-08', 'ses-09', 'ses-10', 'ses-11', 'ses-12']


In [6]:
verboseprint('*'*79)
verboseprint('Tasks: %s\n, Subjects: %s\n, derivatives_dir: %s\n, data_dir: %s' % 
     (tasks, subjects, derivatives_dir, data_dir))
verboseprint('*'*79)

*******************************************************************************
Tasks: ['cuedTS', 'directedForgetting', 'flanker', 'goNogo', 'nBack', 'shapeMatching', 'spatialTS', 'stopSignal', 'stopSignalWDirectedForgetting', 'stopSignalWFlanker', 'directedForgettingWFlanker']
, Subjects: ['s03', 's10', 's19', 's29', 's43']
, derivatives_dir: /oak/stanford/groups/russpold/data/network_grant/discovery_BIDS/derivatives
, data_dir: /oak/stanford/groups/russpold/data/network_grant/discovery_BIDS
*******************************************************************************


# Set up Nodes

### Run analysis

gather the files for each task within each subject


In [7]:
print(join(data_dir, ))

/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS


In [7]:
to_run = []
for subject_id in subjects:
    for session in ses:
        for task in tasks:
            verboseprint('Setting up %s, %s' % (subject_id, task))
            if task == 'goNogo':
                event_files = glob(data_dir+"/sub-"+subject_id+'/'+session+'/func/*goNogo*events.tsv')
                if len(event_files) != 0:
                    event_df = pd.read_csv(event_files[0], sep='\t')
                    if event_df['trial_type'].str.contains('nogo_failure').any():
                        task = 'goNogo_nogo_failure'
                    else:
                        task = 'goNogo'
            files = get_first_level_objs(subject_id, session, task, first_level_dir, beta=False, regress_rt=regress_rt)
            if len(files) == 0 or args.overwrite:
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore",category=DeprecationWarning)
                    warnings.filterwarnings("ignore",category=UserWarning)
                    subjinfo = make_first_level_obj(subject_id, session, task, fmriprep_dir, 
                                                    data_dir, first_level_dir, TR, regress_rt=regress_rt, a_comp_cor=a_comp_cor)
                if subjinfo is not None:
                    to_run.append(subjinfo)

Setting up s03, cuedTS
Missing MRI files for s03: ses-01 cuedTS
Setting up s03, directedForgetting
Missing MRI files for s03: ses-01 directedForgetting
Setting up s03, flanker
Missing MRI files for s03: ses-01 flanker
Setting up s03, goNogo
Setting up s03, nBack
['/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS/derivatives/SOBC_analysis/sub-s03/ses-01/func/sub-s03_ses-01_task-nBack_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']
Missing event files for s03: ses-01 nBack
Setting up s03, shapeMatching
Setting up s03, spatialTS
Setting up s03, stopSignal
Missing MRI files for s03: ses-01 stopSignal
Setting up s03, stopSignalWDirectedForgetting
Missing MRI files for s03: ses-01 stopSignalWDirectedForgetting
Setting up s03, stopSignalWFlanker
Missing MRI files for s03: ses-01 stopSignalWFlanker
Setting up s03, directedForgettingWFlanker
Missing MRI files for s03: ses-01 directedForgettingWFlanker
Setting up s03, cuedTS
['/oak/stanford/groups/russpold/data/network_grant/

In [8]:
to_run

### Run model fit

generate the glm and fit the timeseries data to it

In [9]:
for subjinfo in to_run:
    verboseprint(subjinfo.ID)
    verboseprint('** fitting model')
    fmri_glm = FirstLevelModel(TR, 
                           subject_label = subjinfo.ID,
                           mask_img=subjinfo.mask,
                           noise_model='ar1',
                           standardize=False, 
                           hrf_model='spm',
                           drift_model='cosine',
                           n_jobs=1
                          )
    
    if args.design_matrix:
        verboseprint('** saving')
        save_first_level_obj(subjinfo, first_level_dir, False)
        subjinfo.export_design(first_level_dir)
        subjinfo.export_events(first_level_dir)
    else:
        out = fmri_glm.fit(subjinfo.func, design_matrices=subjinfo.design)
        subjinfo.fit_model = out

        verboseprint('** saving')
        save_first_level_obj(subjinfo, first_level_dir, True)
        subjinfo.export_design(first_level_dir)
        subjinfo.export_events(first_level_dir)


s03_ses-02_cuedTS
** fitting model


/home/users/sjshim/miniconda3/envs/fmri_analysis/lib/python3.9/site-packages/nistats/first_level_model.py:73: UserWarning: Mean values of 0 observed.The data have probably been centered.Scaling might not work as expected
  warn('Mean values of 0 observed.'


** saving
s03_ses-02_directedForgetting
** fitting model
** saving
s03_ses-02_flanker
** fitting model
** saving
s03_ses-02_nBack
** fitting model
** saving
s03_ses-02_stopSignal
** fitting model
** saving
s03_ses-03_nBack
** fitting model
** saving
s03_ses-03_shapeMatching
** fitting model
** saving
s03_ses-03_spatialTS
** fitting model
** saving
s03_ses-04_cuedTS
** fitting model
** saving
s03_ses-04_directedForgetting
** fitting model
** saving
s03_ses-04_flanker
** fitting model
** saving
s03_ses-04_stopSignal
** fitting model


KeyboardInterrupt: 